In [10]:
from transformers import BertModel, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset
import nlp
import torch
from torch import nn as nn
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
RANDOM_SEED = 50
MAX_LEN = 512
EPOCHS=4
if torch.cuda.device_count() > 1:
    BATCH_SIZE = torch.cuda.device_count() * 8
else:
    BATCH_SIZE = 8

In [3]:
dataset = load_dataset('csv', data_files='./reviews.csv', split='train')
shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']

Using custom data configuration default


In [4]:
def sample(ds, ratio):
    return random.sample(range(0, len(ds)), int(len(ds) * ratio)) 

train_dataset = train_dataset.select(indices=sample(train_dataset, 0.1))
test_dataset = test_dataset.select(indices=sample(test_dataset, 0.1))
val_dataset = val_dataset.select(indices=sample(val_dataset, 0.1))

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

def tokenize(batch):
    return tokenizer(batch['content'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [19]:

class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-chinese')
        self.drop = nn.Dropout(p=0.2)
        # self.lstm = nn.LSTM
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [29]:
data = train_dataset[0]
input_ids = data['input_ids'].unsqueeze(dim=0)
attention_mask = data['attention_mask'].unsqueeze(dim=0)
attention_mask.shape

torch.Size([1, 512])

In [30]:
class_names = ['negative', 'positive']
model = SentimentClassifier(len(class_names))

In [37]:
bert_model = BertModel.from_pretrained('bert-base-chinese')
h, o = bert_model(input_ids=input_ids, attention_mask=attention_mask)
print(o.shape)

dropout = nn.Dropout(p=0.3)
o2 = dropout(o)
print(o2.shape)

torch.Size([1, 768])
torch.Size([1, 768])
